In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: konggedzu (imucs). Use `wandb login --relogin` to force relogin


True

In [2]:
sweep_config = {
    'method': 'random'
    }

In [3]:
metric = {
    'name': 'Grand Mean',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [4]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
        },

    'learning_rate': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
        },

    'batch_size': {
        'values': [256, 512, 1024, 2048, 4096]
        },

    'data_augmentation_multiple': {
        'values': [1, 3, 5, 7, 9, 11, 13, 15]
        }
}


In [5]:
sweep_config['parameters'] = parameters_dict

In [6]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Grand Mean'},
 'parameters': {'batch_size': {'values': [256, 512, 1024, 2048, 4096]},
                'data_augmentation_multiple': {'values': [1,
                                                          3,
                                                          5,
                                                          7,
                                                          9,
                                                          11,
                                                          13,
                                                          15]},
                'dropout': {'values': [0.01,
                                       0.1,
                                       0.2,
                                       0.3,
                                       0.4,
                                       0.5,
                                       0.6,
                                       0.7,
      

In [7]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-sub_loc")

Create sweep with ID: s6im0w89
Sweep URL: https://wandb.ai/imucs/pytorch-sweeps-sub_loc/sweeps/s6im0w89


In [8]:
import pandas as pd
from dataAug.tools import dataAugForTestSMOTE, selectSameLabel, KG_MLAug, test_aug
from classify.processTools import processRealData
import torch
from classify.classify_for_parameter_adjustment import ModelClassify
from classify.targeTools import testThresholdFive, Accuracy

In [9]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/DPC_T5_578_bhd.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [10]:
from torch import optim


def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

In [11]:
smote_multiple = {}

In [12]:
from torch.utils.data import TensorDataset, DataLoader
import time
import datetime

def train_and_val(config=None):
    with wandb.init(config=config):
        config = wandb.config
        multi_label_samples = labels_pd[(labels_pd.sum(axis=1) >= 2)]
        multi_label_indices = multi_label_samples.index
        multi_features_samples = feature_pd.loc[multi_label_indices]
        multi_features_samples = multi_features_samples.drop([43, 522], axis=0)
        multi_label_samples = multi_label_samples.drop([43, 522], axis=0)
        multi_label_samples = multi_label_samples.drop(columns='plastoglobule')
        new_X, new_y = KG_MLAug(multi_features_samples, multi_label_samples, config.data_augmentation_multiple)
        test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/DPC_T5_test_mutil_label_122_finally.csv")

        G_feature, G_label = dataAugForTestSMOTE(feature_pd, labels_pd, config.data_augmentation_multiple, 1424)
        R_feature, R_label = processRealData(feature_pd, labels_pd, feature_num=1424)

        # 合并生成的多标签
        G_feature = pd.concat([G_feature, new_X], axis=0)
        G_label = pd.concat([G_label, new_y], axis=0)

        # new_testX, new_testY = test_aug(test_df, config.data_augmentation_multiple)
        # G_feature = pd.concat([G_feature, new_testX], axis=0)
        # G_label = pd.concat([G_label, new_testY], axis=0)


        train_feature = pd.concat([R_feature, G_feature], axis=0)
        train_label = pd.concat([R_label, G_label], axis=0)

        datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
        batch_size = config.batch_size
        dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

        datasetTest = TensorDataset(torch.tensor(test_df.iloc[:,:1424].values), torch.tensor(test_df.iloc[:,-4:].values))

        batch_size = len(datasetTest)
        dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

        model = ModelClassify(config.dropout)
        criterion = torch.nn.BCELoss()
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate)

        import warnings
        warnings.filterwarnings("ignore")
        GMList = {}

        for epoch in range(500):
            model.train()
            total_loss = 0.0
            for idx, data in enumerate(dataloaderTrain, 0):
                inputs, labels = data
                labels = labels.float()
                inputs = inputs.float()
                out = model(inputs)
                loss = criterion(out, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(dataloaderTrain)
            threshold = 0.5
            labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
            print(f"Epoch [{epoch+1}/{300}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
            GMScore1 = testThresholdFive(epoch, model, dataloaderTest)
            GMList[epoch] = GMScore1
        t = time.time()
        torch.save(model, "/home/kongge/projects/new_protT5/model/model_" + str(t) +".pt")
        best_key = max(GMList, key=GMList.get)
        best_value = GMList[best_key]
        print(t)
        smote_multiple[int(t)] = best_value

        wandb.log({"Grand Mean": best_value[0].item()})


In [ ]:
wandb.agent(sweep_id, train_and_val, count=50)

wandb: Agent Starting Run: 80fpmrtp with config:
wandb: 	batch_size: 2048
wandb: 	data_augmentation_multiple: 1
wandb: 	dropout: 0.3
wandb: 	learning_rate: 0.08187565970388855
wandb: 	optimizer: sgd


/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (398) in class 0 will be larger than the number of samples in the majority class (class #1 -> 379)
  f"After over-sampling, the number of samples ({n_samples})"
/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (466) in class 0 will be larger than the number of samples in the majority class (class #1 -> 345)
  f"After over-sampling, the number of samples ({n_samples})"


Epoch [1/300], Average Loss: 0.7286, ACC: 0.24296623794212238
epoch:0, bestThreshold:[0.01, 0.01, 0.01, 0.01], GM:0.39131155610084534, OAA:0.0, ACC:0.26844262295081966, F1:0.41967248916625977
Epoch [2/300], Average Loss: 0.6997, ACC: 0.29320739549839175
epoch:1, bestThreshold:[0.01, 0.49, 0.01, 0.01], GM:0.4280874729156494, OAA:0.0, ACC:0.32991803278688525, F1:0.4887979328632355
Epoch [3/300], Average Loss: 0.6555, ACC: 0.3733922829581985
epoch:2, bestThreshold:[0.01, 0.48, 0.01, 0.01], GM:0.41196727752685547, OAA:0.0, ACC:0.3012295081967213, F1:0.4573773443698883
Epoch [4/300], Average Loss: 0.6105, ACC: 0.4529742765273311
epoch:3, bestThreshold:[0.46, 0.48, 0.01, 0.01], GM:0.4328414797782898, OAA:0.0, ACC:0.3463114754098361, F1:0.5043715238571167
Epoch [5/300], Average Loss: 0.5669, ACC: 0.5277331189710616
epoch:4, bestThreshold:[0.01, 0.45, 0.49, 0.01], GM:0.39049187302589417, OAA:0.0, ACC:0.27868852459016386, F1:0.42786914110183716
Epoch [6/300], Average Loss: 0.5347, ACC: 0.565313

Grand Mean,▁
Grand Mean,0.82678


wandb: Agent Starting Run: peklros7 with config:
wandb: 	batch_size: 256
wandb: 	data_augmentation_multiple: 15
wandb: 	dropout: 0.3
wandb: 	learning_rate: 0.07477868877531503
wandb: 	optimizer: adam


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: s9c4zswq with config:
wandb: 	batch_size: 2048
wandb: 	data_augmentation_multiple: 11
wandb: 	dropout: 0.01
wandb: 	learning_rate: 0.03911910285896863
wandb: 	optimizer: sgd


Epoch [1/300], Average Loss: 0.5499, ACC: 0.5833333333333333
epoch:0, bestThreshold:[0.01, 0.01, 0.5, 0.01], GM:0.3930055499076843, OAA:0.0, ACC:0.27390710382513667, F1:0.4254102110862732
Epoch [2/300], Average Loss: 0.3255, ACC: 0.875
epoch:1, bestThreshold:[0.49, 0.42, 0.44, 0.47], GM:0.39702191948890686, OAA:0.0, ACC:0.3128415300546446, F1:0.4598362147808075
Epoch [3/300], Average Loss: 0.2454, ACC: 0.9166666666666666
epoch:2, bestThreshold:[0.42, 0.35, 0.35, 0.48], GM:0.5655190944671631, OAA:0.319672131147541, ACC:0.5464480874316939, F1:0.6254098415374756
Epoch [4/300], Average Loss: 0.1818, ACC: 0.9166666666666666
epoch:3, bestThreshold:[0.29, 0.26, 0.29, 0.5], GM:0.5459016561508179, OAA:0.36885245901639346, ACC:0.528688524590164, F1:0.5874317288398743
Epoch [5/300], Average Loss: 0.1584, ACC: 0.9166666666666666
epoch:4, bestThreshold:[0.2, 0.23, 0.22, 0.45], GM:0.6557377576828003, OAA:0.48360655737704916, ACC:0.644808743169399, F1:0.6994535326957703
Epoch [6/300], Average Loss: 0

Grand Mean,▁
Grand Mean,0.80383


wandb: Agent Starting Run: r9orpeh5 with config:
wandb: 	batch_size: 512
wandb: 	data_augmentation_multiple: 5
wandb: 	dropout: 0.7
wandb: 	learning_rate: 0.059201619470146495
wandb: 	optimizer: adam


Epoch [1/300], Average Loss: 0.5758, ACC: 0.4605855855855856
epoch:0, bestThreshold:[0.01, 0.03, 0.17, 0.56], GM:0.4516393542289734, OAA:0.3114754098360656, ACC:0.4426229508196721, F1:0.4877049922943115
Epoch [2/300], Average Loss: 0.4079, ACC: 0.43355855855855857
epoch:1, bestThreshold:[0.03, 0.07, 0.48, 0.41], GM:0.4964207708835602, OAA:0.29508196721311475, ACC:0.4815573770491804, F1:0.546994686126709
Epoch [3/300], Average Loss: 0.3353, ACC: 0.7188438438438439
epoch:2, bestThreshold:[0.07, 0.08, 0.34, 0.69], GM:0.6415300369262695, OAA:0.47540983606557374, ACC:0.6311475409836066, F1:0.6830600500106812
Epoch [4/300], Average Loss: 0.2601, ACC: 0.7462462462462462
epoch:3, bestThreshold:[0.09, 0.6, 0.43, 0.67], GM:0.6901639103889465, OAA:0.5983606557377049, ACC:0.6844262295081968, F1:0.7131146192550659
Epoch [5/300], Average Loss: 0.2166, ACC: 0.789039039039039
epoch:4, bestThreshold:[0.11, 0.87, 0.36, 0.41], GM:0.660655677318573, OAA:0.5409836065573771, ACC:0.6530054644808743, F1:0.691

In [ ]:
smote_multiple